<a href="https://colab.research.google.com/github/gabrielgranados97/Tareas-IA/blob/main/Tarea5_IA_LeNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Gabriel Fernando Granados Álvarez**

**Universidad Nacional de Colombia**

**Tópicos IA** 

Tarea n°5

Modelo LeNet

In [1]:
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn

import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from torch.autograd import Variable
from sklearn.metrics import confusion_matrix

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [2]:
train_set = torchvision.datasets.FashionMNIST("/mnt/storage/Datasets", download=True, transform=
                                                transforms.Compose([transforms.ToTensor()]))
test_set = torchvision.datasets.FashionMNIST("/mnt/storage/Datasets", download=True, train=False, transform=
                                               transforms.Compose([transforms.ToTensor()])) 

  0%|          | 0/26421880 [00:00<?, ?it/s]

Extracting /mnt/storage/Datasets/FashionMNIST/raw/train-images-idx3-ubyte.gz to /mnt/storage/Datasets/FashionMNIST/raw



  0%|          | 0/29515 [00:00<?, ?it/s]

Extracting /mnt/storage/Datasets/FashionMNIST/raw/train-labels-idx1-ubyte.gz to /mnt/storage/Datasets/FashionMNIST/raw



  0%|          | 0/4422102 [00:00<?, ?it/s]

Extracting /mnt/storage/Datasets/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to /mnt/storage/Datasets/FashionMNIST/raw



  0%|          | 0/5148 [00:00<?, ?it/s]

Extracting /mnt/storage/Datasets/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to /mnt/storage/Datasets/FashionMNIST/raw



In [3]:
train_loader = torch.utils.data.DataLoader(train_set, 
                                           batch_size=100)
test_loader = torch.utils.data.DataLoader(test_set,
                                          batch_size=100)

batch = next(iter(test_loader))
images, labels = batch
print(type(images), type(labels))
print(images.shape, labels.shape)

<class 'torch.Tensor'> <class 'torch.Tensor'>
torch.Size([100, 1, 28, 28]) torch.Size([100])


In [4]:
class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 6, kernel_size=5, padding=2), 
            nn.Sigmoid(),
            nn.AvgPool2d(kernel_size=2, stride=2)
        )
    
        self.layer2 = nn.Sequential(
            nn.Conv2d(6, 16, kernel_size=5),
            nn.Sigmoid(),
            nn.AvgPool2d(kernel_size=2, stride=2)
        )
    
        self.fc1 = nn.Linear(16*5*5, 120)
        self.sigmoid1 = nn.Sigmoid()
        self.fc2 = nn.Linear(120, 84)
        self.sigmoid2 = nn.Sigmoid()
        self.fc3 = nn.Linear(84, 10)
    
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1)
        out = self.fc1(out)
        out = self.sigmoid1(out)
        out = self.fc2(out)
        out = self.sigmoid2(out)
        out = self.fc3(out)

        return out

In [5]:
model = LeNet()
model.to(device)

error = nn.CrossEntropyLoss()

learning_rate = 1.0
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
print(model)

LeNet(
  (layer1): Sequential(
    (0): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): Sigmoid()
    (2): AvgPool2d(kernel_size=2, stride=2, padding=0)
  )
  (layer2): Sequential(
    (0): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
    (1): Sigmoid()
    (2): AvgPool2d(kernel_size=2, stride=2, padding=0)
  )
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (sigmoid1): Sigmoid()
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (sigmoid2): Sigmoid()
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [6]:
num_epochs = 10
count = 0
loss_list = []
iteration_list = []
accuracy_list = []

predictions_list = []
labels_list = []

for epoch in range(num_epochs):
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
    
        train = Variable(images.view(100, 1, 28, 28))
        labels = Variable(labels)
        
        outputs = model(train)
        loss = error(outputs, labels)
        
        optimizer.zero_grad()
        
        loss.backward()
        
        optimizer.step()
    
        count += 1
    
    
        if not (count % 50):
            total = 0
            correct = 0
        
            for images, labels in test_loader:
                images, labels = images.to(device), labels.to(device)
                labels_list.append(labels)
            
                test = Variable(images.view(100, 1, 28, 28))
            
                outputs = model(test)
            
                predictions = torch.max(outputs, 1)[1].to(device)
                predictions_list.append(predictions)
                correct += (predictions == labels).sum()
            
                total += len(labels)
            
            accuracy = correct * 100 / total
            loss_list.append(loss.data)
            iteration_list.append(count)
            accuracy_list.append(accuracy)
        
        if not (count % 500):
            print("Iteration: {}, Loss: {}, Accuracy: {}%".format(count, loss.data, accuracy))

Iteration: 500, Loss: 2.307831048965454, Accuracy: 10.0%
Iteration: 1000, Loss: 2.2943944931030273, Accuracy: 10.0%
Iteration: 1500, Loss: 2.3061020374298096, Accuracy: 10.0%
Iteration: 2000, Loss: 2.3004250526428223, Accuracy: 10.0%
Iteration: 2500, Loss: 2.3000376224517822, Accuracy: 10.0%
Iteration: 3000, Loss: 2.296536445617676, Accuracy: 18.950000762939453%
Iteration: 3500, Loss: 0.8879095315933228, Accuracy: 61.470001220703125%
Iteration: 4000, Loss: 0.6860784888267517, Accuracy: 73.20999908447266%
Iteration: 4500, Loss: 0.5434072017669678, Accuracy: 75.08999633789062%
Iteration: 5000, Loss: 0.7215997576713562, Accuracy: 74.9800033569336%
Iteration: 5500, Loss: 0.538689911365509, Accuracy: 79.66999816894531%
Iteration: 6000, Loss: 0.4246789216995239, Accuracy: 82.54000091552734%


In [7]:
class LeNet_Norm(nn.Module):
    def __init__(self):
        super(LeNet_Norm, self).__init__()
        
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 6, kernel_size=5, padding=2), 
            nn.Sigmoid(),
            nn.BatchNorm2d(6),
            nn.AvgPool2d(kernel_size=2, stride=2)
        )
    
        self.layer2 = nn.Sequential(
            nn.Conv2d(6, 16, kernel_size=5),
            nn.BatchNorm2d(16),
            nn.Sigmoid(),
            nn.AvgPool2d(kernel_size=2, stride=2)
        )
    
        self.fc1 = nn.Linear(16*5*5, 120)
        self.batchnorm1 = nn.BatchNorm1d(120)
        self.sigmoid1 = nn.Sigmoid()
        self.fc2 = nn.Linear(120, 84)
        self.batchnorm2 = nn.BatchNorm1d(84)
        self.sigmoid2 = nn.Sigmoid()
        self.fc3 = nn.Linear(84, 10)
    
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1)
        out = self.fc1(out)
        out = self.batchnorm1(out)
        out = self.sigmoid1(out)
        out = self.fc2(out)
        out = self.batchnorm2(out)
        out = self.sigmoid2(out)
        out = self.fc3(out)

        return out

In [8]:
model = LeNet_Norm()
model.to(device)

error = nn.CrossEntropyLoss()

learning_rate = 1.0
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
print(model)

LeNet_Norm(
  (layer1): Sequential(
    (0): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): Sigmoid()
    (2): BatchNorm2d(6, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): AvgPool2d(kernel_size=2, stride=2, padding=0)
  )
  (layer2): Sequential(
    (0): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
    (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): Sigmoid()
    (3): AvgPool2d(kernel_size=2, stride=2, padding=0)
  )
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (batchnorm1): BatchNorm1d(120, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (sigmoid1): Sigmoid()
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (batchnorm2): BatchNorm1d(84, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (sigmoid2): Sigmoid()
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [9]:
num_epochs = 10
count = 0
loss_list = []
iteration_list = []
accuracy_list = []

predictions_list = []
labels_list = []

for epoch in range(num_epochs):
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
    
        train = Variable(images.view(100, 1, 28, 28))
        labels = Variable(labels)
        
        outputs = model(train)
        loss = error(outputs, labels)
        
        optimizer.zero_grad()
        
        loss.backward()
        
        optimizer.step()
    
        count += 1

    
        if not (count % 50):   
            total = 0
            correct = 0
        
            for images, labels in test_loader:
                images, labels = images.to(device), labels.to(device)
                labels_list.append(labels)
            
                test = Variable(images.view(100, 1, 28, 28))
            
                outputs = model(test)
            
                predictions = torch.max(outputs, 1)[1].to(device)
                predictions_list.append(predictions)
                correct += (predictions == labels).sum()
            
                total += len(labels)
            
            accuracy = correct * 100 / total
            loss_list.append(loss.data)
            iteration_list.append(count)
            accuracy_list.append(accuracy)
        
        if not (count % 500):
            print("Iteration: {}, Loss: {}, Accuracy: {}%".format(count, loss.data, accuracy))

Iteration: 500, Loss: 0.5087994933128357, Accuracy: 82.79000091552734%
Iteration: 1000, Loss: 0.3449772596359253, Accuracy: 85.86000061035156%
Iteration: 1500, Loss: 0.3365040123462677, Accuracy: 86.75%
Iteration: 2000, Loss: 0.43012839555740356, Accuracy: 85.70999908447266%
Iteration: 2500, Loss: 0.2099885791540146, Accuracy: 87.76000213623047%
Iteration: 3000, Loss: 0.23050779104232788, Accuracy: 88.47000122070312%
Iteration: 3500, Loss: 0.40944427251815796, Accuracy: 88.55000305175781%
Iteration: 4000, Loss: 0.2866641879081726, Accuracy: 88.9800033569336%
Iteration: 4500, Loss: 0.20674943923950195, Accuracy: 89.05000305175781%
Iteration: 5000, Loss: 0.30335429310798645, Accuracy: 87.83000183105469%
Iteration: 5500, Loss: 0.171071857213974, Accuracy: 89.02999877929688%
Iteration: 6000, Loss: 0.22070661187171936, Accuracy: 89.37000274658203%
